# Shapes of variables in snow components

**Problem:** There's an issue with setting the values of `h_snow` or `rho_snow` after initialization because of a conflict with the shapes of internal variables in the snow components. In each case, `set_value` works, but a failure occurs on calling `update`; in particular, on line 467, in `update_swe`, in **snow_base.py**.

The shape conflict doesn't occur when setting `h_swe` or `SM`.

Import the Babel-wrapped `SnowDegreeDay` component and create an instance.

In [ ]:
from cmt.components import SnowDegreeDay
sno = SnowDegreeDay()

Initialize the component.

In [ ]:
sno.initialize('./input/snow_degree_day-1.cfg')

Get the model time step (which has been broadcast over the grid).

In [ ]:
time_step = sno.get_value('model__time_step').max()
print 'Time step = {} s'.format(time_step)

Assign a new value to the snowpack depth. (Note that if this step is omitted, the model will update successfully.)

In [ ]:
sno.set_value('snowpack__depth', 0.6)  # [m]

Check the new snowpack depth.

In [ ]:
h_snow = sno.get_value('snowpack__depth')
print h_snow.shape
h_snow

Advance the model.

In [ ]:
sno.update(time_step)

**Note:** I believe I've identified the issue. TopoFlow uses Numpy arrays for all internal variables, including scalars. At some point (my hunch is in the Babelized `set_value` method), a one-element list, instead of a single scalar value, was set as data for these variables. This caused the shape mismatches. I fixed this problem by adding a check in the `set_value` method that we grafted onto TopoFlow:

```python
if src.shape == (1,):
  src = np.array(src[0])
```

This converts the one-element list into a scalar.

I think this fix is robust.

Finalize the component.

In [ ]:
sno.finalize()

**Result:** Problem solved, assuming my fix holds up.

For reference, here's the stack trace of the error that had been occurring:

```
An error occured inside a babel wrapper. Here is the traceback.
Traceback (most recent call last):
  File "/home/csdms/wmt/topoflow.0/csdms/install/lib/python2.7/site-packages/csdms/SnowDegreeDay_Impl.py", line 696, in _wrap
    return func(*args)
  File "/home/csdms/wmt/topoflow.0/csdms/install/lib/python2.7/site-packages/csdms/SnowDegreeDay_Impl.py", line 107, in update
    self._model.update_until(time_interval)
  File "/home/csdms/wmt/topoflow.0/cache/topoflow-12d38ce246/topoflow/utils/BMI_base.py", line 1104, in update_until
    self.update()
  File "/home/csdms/wmt/topoflow.0/cache/topoflow-12d38ce246/topoflow/components/snow_base.py", line 238, in update
    self.update_swe()
  File "/home/csdms/wmt/topoflow.0/cache/topoflow-12d38ce246/topoflow/components/snow_base.py", line 467, in update_swe
    self.h_swe -= dh2_swe
ValueError: non-broadcastable output operand with shape () doesn't match the broadcast shape (1,)
```